# ────────────────────────────────────────────────────────────────────────
# 1 ▸ Imports
# ────────────────────────────────────────────────────────────────────────

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


Bloch-vector dynamics under a (possibly detuned) drive — dimensionless version
==============================================================================
This single script is *fully self-contained*
We keep everything in **dimensionless time**

    for expectation value : τ = T / ω_21
    for plot we have: t = ω_21 · τ

so *no numerical value for ω_21 is ever needed or used.*


# ────────────────────────────────────────────────────────────────────────
# 2 ▸ User-editable parameters
# ────────────────────────────────────────────────────────────────────────

In [ ]:
chi_1 = 0.1  # Normalised coupling strength gama_tilde / ω_21

# Study cases: each dict must contain at least "chi_2".
# Optional keys "label" and "color" simply control plotting aesthetics.
cases = [
    {"chi_2": 1.0, "label": r"Resonance  $\chi_2 = 1.0$", "color": "dodgerblue"},
    {"chi_2": 0.8, "label": r"Non-Resonance  $\chi_2 = 0.8$", "color": "lime"},
    {"chi_2": 0.6, "label": r"Non-Resonance  $\chi_2 = 0.6$", "color": "orangered"},
]

N_SAMPLES = 400  # number of τ grid points
T = 60.0

### Translating the formulas
| symbol               | definition                | dimensionless form used in code                                                                                                              |
|----------------------| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------- |
| $\tilde\gamma$       | coupling strength         | $\tilde\gamma = \chi_1\,\omega_{21}$                                                                                                              |
| $\omega$             | drive frequency           | $\omega   = \chi_2\,\omega_{21}$                                                                                                             |
| $\tilde\omega$       | detuning                  | $\tilde\omega = \omega-\omega_{21}=\omega_{21}\,(\chi_2-1)$                                                                                  |
| $\Omega$           | generalized frequency | $\displaystyle\Omega=\sqrt{\tilde\gamma^{\,2}+\frac{\tilde\omega^{\,2}}{4}}=\omega_{21}\sqrt{\chi_1^{2}+\left(\frac{\chi_2-1}{2}\right)^{\!2}}$ |
| $r_\alpha,\;r_\beta$ | weighting factors         | $\displaystyle r_\alpha=\frac{\Omega_R+\tilde\omega/2}{\tilde\gamma},\qquad r_\beta=\frac{-\Omega_R+\tilde\omega/2}{\tilde\gamma}$                 |


This function calculates the expected values of the spin components ($S_x$, $S_y$, $S_z$)
and the total Bloch vector magnitude for given drive parameters and time points.

- `t`: array of time points
- `omega_21`: system's transition frequency
- `gamma_tilde`: coupling strength
- `chi_2`: drive frequency ratio
- `Sx_R`, `Sy_R`, `Sz_R`: arrays of expectation values (rotating/Dirac frame) for each time
- `S_magnitude`: array of total spin magnitudes (should be ~1 for pure state)

Returns **laboratory-frame** Bloch components:

$$
\begin{align*}
S_x^\text{lab}(t) &= \cos(\omega_0 t)\, S_x^R(t) - \sin(\omega_0 t)\, S_y^R(t) \\
S_y^\text{lab}(t) &= \sin(\omega_0 t)\, S_x^R(t) + \cos(\omega_0 t)\, S_y^R(t) \\
S_z^\text{lab}(t) &= S_z^R(t) \\
\end{align*}
$$

where $\omega_0 = \omega_{21}$ and $(S_x^R, S_y^R, S_z^R)$ are the Dirac-picture (rotating-frame) components.

# ────────────────────────────────────────────────────────────────────────
# 3 ▸ Analytic Bloch-vector solution
# ────────────────────────────────────────────────────────────────────────

In [ ]:
def expectation(tau: np.ndarray, chi1: float, chi2: float):
    """Return Sx, Sy, Sz, |⟨S⟩| arrays.

    Parameters
    ----------
    tau  : dimensionless time array because τ = T / ω_21
    chi1 : drive strength gama_tilde / ω_21
    chi2 : drive-frequency / system-frequency ratio (ω / ω_21)
    """
    # Generalised Rabi frequency
    omega = np.sqrt(chi1 ** 2 + ((chi2 - 1) / 2) ** 2)  # omega = ω_21 * np.sqrt(chi_1 ** 2 + ((chi_2 - 1) / 2) ** 2)

    # Dimensionless detuning
    omega_tilde = chi2 - 1  # omega_tilde = ω_21 * (chi_2 - 1)

    Sx_R = (-(chi1 / omega) * np.sin(2 * omega * tau) * np.sin(omega_tilde * tau)
            + (chi1 * omega_tilde / omega ** 2) * np.sin(omega * tau) ** 2 * np.cos(omega_tilde * tau))
    Sy_R = (-(chi1 / omega) * np.sin(2 * omega * tau) * np.cos(omega_tilde * tau)
            - (chi1 * omega_tilde / omega ** 2) * np.sin(omega * tau) ** 2 * np.sin(omega_tilde * tau))
    Sz_R = 1 - (2 * chi1 ** 2 / omega ** 2) * np.sin(omega * tau) ** 2

    # Dirac pic back to the lab frame:
    cos_tau, sin_tau = np.cos(tau), np.sin(tau)
    Sx_lab = cos_tau * Sx_R - sin_tau * Sy_R
    Sy_lab = sin_tau * Sx_R + cos_tau * Sy_R
    Sz_lab = Sz_R

    # (should remain ≈ 1 at all times)
    S_mag = np.sqrt(Sx_lab ** 2 + Sy_lab ** 2 + Sz_lab ** 2)
    return Sx_lab, Sy_lab, Sz_lab, S_mag

### Expectation values of the spin components (divided by ℏ/2)

$$
\begin{aligned}
\langle S_x(t)\rangle &=
    \frac{\tilde{\gamma}^{\,2}}{\Omega^{2}}
    \,\sin(\Omega t)\!
    \bigl[
        r_\beta\,\sin\!\bigl((\Omega+\tilde{\omega})t\bigr)
      + r_\alpha\,\sin\!\bigl((\Omega-\tilde{\omega})t\bigr)
    \bigr],\\[6pt]
\end{aligned}
$$

$$
\langle S_x(t)\rangle \;=\;
-\,\frac{\tilde{\gamma}}{\Omega}\,
    \sin\!\bigl(2\Omega t\bigr)\,\sin(\tilde{\omega} t)
\;+\;
\frac{\tilde{\gamma}\,\tilde{\omega}}{\Omega^{2}}\,
    \sin^{2}(\Omega t)\,\cos(\tilde{\omega} t)
$$

$$
\begin{aligned}
\langle S_y(t)\rangle &=
    \frac{\tilde{\gamma}^{\,2}}{\Omega^{2}}
    \,\sin(\Omega t)\!
    \bigl[
        r_\beta\,\cos\!\bigl((\Omega+\tilde{\omega})t\bigr)
      - r_\alpha\,\cos\!\bigl((\Omega-\tilde{\omega})t\bigr)
    \bigr],\\[6pt]
\end{aligned}
$$

$$
\langle S_y(t)\rangle \;=\;
-\,\frac{\tilde{\gamma}}{\Omega}\,
    \sin\!\bigl(2\Omega t\bigr)\,\cos(\tilde{\omega} t)
\;-\;
\frac{\tilde{\gamma}\,\tilde{\omega}}{\Omega^{2}}\,
    \sin^{2}(\Omega t)\,\sin(\tilde{\omega} t)
$$

$$
\langle S_z(t)\rangle \;=\;
1 \;-\;
\frac{2\,\tilde{\gamma}^{2}}{\Omega^{2}}\,
    \sin^{2}(\Omega t)
$$

### total spin magnitude:

$$
\bigl|\langle \mathbf{S}(t)\rangle\bigr|
  \;=\;
  \sqrt{\,
        \langle S_x(t)\rangle^{2}
      + \langle S_y(t)\rangle^{2}
      + \langle S_z(t)\rangle^{2}\,}.
$$








# ──────────────────────────────────────────────────────────────────────────
# 4 ▸ Compute trajectories for every case
# ──────────────────────────────────────────────────────────────────────────

In [ ]:
def compute_bloch_evolution(
        chi1: float,
        cases: list,
        n_samples: int = N_SAMPLES,
        T: float = T,
):
    # Build the master τ axis
    tau = np.linspace(0.0, T, n_samples)

    results = []
    for case in cases:
        chi2 = case["chi_2"]
        Sx, Sy, Sz, S_mag = expectation(tau, chi1, chi2)

        # Bundle everything in a dict for convenience later
        results.append({
            "chi_2": chi2,
            "Sx_lab": Sx,
            "Sy_lab": Sy,
            "Sz_lab": Sz,
            "S_magnitude": S_mag,
            "label": case.get("label", f"chi_2 = {chi2:.1g}"),
            "color": case.get("color", None),
        })
    return tau, results


# ────────────────────────────────────────────────────────────────────────
# 5 ▸ Numerical run
# ────────────────────────────────────────────────────────────────────────

In [ ]:
tau, results = compute_bloch_evolution(chi_1, cases)
for r in results:
    print(f" chi_2 = {r['chi_2']:<4} mean |⟨S⟩| = {np.mean(r['S_magnitude']):.1f}")

# ────────────────────────────────────────────────────────────────────────
# 6 ▸ Static figure: time-series + empty Bloch sphere
# ────────────────────────────────────────────────────────────────────────

In [ ]:
# Creates a 2×2 grid of subplots:
#   Notice: we have : t = ω_21 · τ
#   (0,0) Sx(ω_21 . τ)   (0,1) Sy(ω_21 . τ)
#   (1,0) Sz(ω_21 . τ)   (1,1) Bloch sphere (3-D)
# ------------------------------------------------------------------------

fig = plt.figure(figsize=(14, 10))
ax1 = fig.add_subplot(221)  # Sx(ω_21 . τ)
ax2 = fig.add_subplot(222)  # Sy(ω_21 . τ)
ax3 = fig.add_subplot(223)  # Sz(ω_21 . τ)
ax4 = fig.add_subplot(224, projection="3d")  # Bloch sphere

# Plot each component vs  ω_21 · τ :
# for plot we have: t = ω_21 · τ = ω21 · ( T / ω_21 ) = T
for r in results:
    ax1.plot(tau, r["Sx_lab"], label=r["label"], color=r["color"])
    ax2.plot(tau, r["Sy_lab"], label=r["label"], color=r["color"])
    ax3.plot(tau, r["Sz_lab"], label=r["label"], color=r["color"])

# Niceties: labels, titles, grids
for ax, ylabel, title in [
    (ax1, r"$\langle S_x \rangle / \hbar/2$", r"$\langle S_x \rangle / \hbar/2$"" vs ω₍₂₁₎ τ"),
    (ax2, r"$\langle S_y \rangle / \hbar/2$", r"$\langle S_y \rangle / \hbar/2$"" vs ω₍₂₁₎ τ"),
    (ax3, r"$\langle S_z \rangle / \hbar/2$", r"$\langle S_z \rangle / \hbar/2$"" vs ω₍₂₁₎ τ"),
]:
    ax.set_xlabel("Dimensionless time   t = ω₍₂₁₎ τ")
    ax.set_ylabel(ylabel, rotation=0, labelpad=12)
    ax.set_title(title)
    ax.grid(ls="--", alpha=0.7)

# Moving guidelines (one per subplot) — updated during animation
vline1 = ax1.axvline(tau[0], color="k", ls="--")
vline2 = ax2.axvline(tau[0], color="k", ls="--")
vline3 = ax3.axvline(tau[0], color="k", ls="--")

# ────────────────────────────────────────────────────────────────────────
# 7 ▸ Bloch sphere scaffolding
# ────────────────────────────────────────────────────────────────────────

In [ ]:
# Reference sphere
u = np.linspace(0, 2 * np.pi, 40)
v = np.linspace(0, np.pi, 40)
xs = np.outer(np.cos(u), np.sin(v))
ys = np.outer(np.sin(u), np.sin(v))
zs = np.outer(np.ones_like(u), np.cos(v))
ax4.plot_surface(xs, ys, zs, color="bisque", alpha=0.3, rstride=1, cstride=1)
ax4.set_xticks([-1, -0.5, 0, 0.5, 1])
ax4.set_yticks([-1, -0.5, 0, 0.5, 1])
ax4.set_zticks([-1, -0.5, 0, 0.5, 1])
ax4.set_xlabel(r"$\langle S_x \rangle / \hbar/2$");
ax4.set_ylabel(r"$\langle S_y \rangle / \hbar/2$");
ax4.set_zlabel(r"$\langle S_z \rangle / \hbar/2$")
ax4.set_title("Bloch-vector trajectory", pad=10)

traj_lines, vec_lines = [], []
for r in results:
    # 3-D line for the *full path* as it grows with time
    traj, = ax4.plot([], [], [], lw=1.4, color=r["color"], label=r["label"])
    # 3-D thicker line for the *instantaneous state arrow*
    vec, = ax4.plot([], [], [], lw=2.2, color=r["color"])
    traj_lines.append(traj)
    vec_lines.append(vec)
ax4.legend(frameon=False, loc="upper center", ncol=len(results))

# ────────────────────────────────────────────────────────────────────────
# 8 ▸ Animation driver + file output
# ────────────────────────────────────────────────────────────────────────

In [ ]:
def animate(frame: int):
    # Update every case
    for traj, vec, r in zip(traj_lines, vec_lines, results):
        traj.set_data(r["Sx_lab"][:frame + 1], r["Sy_lab"][:frame + 1])
        traj.set_3d_properties(r["Sz_lab"][:frame + 1])
        vec.set_data([0, r["Sx_lab"][frame]], [0, r["Sy_lab"][frame]])
        vec.set_3d_properties([0, r["Sz_lab"][frame]])

    # Slide vertical lines on the time-series plots
    current_tau = tau[frame]
    for vline in (vline1, vline2, vline3):
        vline.set_xdata([current_tau, current_tau])

    return traj_lines + vec_lines + [vline1, vline2, vline3]


# Create animation object
frames = len(tau)
ani = FuncAnimation(fig, animate, frames=frames, interval=100, blit=True)

# Layout and export
plt.tight_layout()
fig.savefig("bloch_vector_evolution_dimless.png", dpi=150)
ani.save("bloch_vector_evolution_dimless.gif", writer="pillow", dpi=150)
print("Saved PNG and GIF (dimensionless version).")